<a href="https://colab.research.google.com/github/adnanelahi/bioimpedance/blob/main/bioimpedance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Impedance Analysis Library v0.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap

In [ ]:
def readAutoLabFile(filename):
  """
  Reads a single measurement text file and converts it into a numpy array

  Parameters
  ----------
  f_path_name_template : str 
    filename along with a path. 
  
  Returns
  ---------
  Z : numpy array
    impedance measurement
  f: numpy array
    frequencies
  """  
  # Read TAB delimited AutoLab generated text file and skip first row
  data = np.genfromtxt(filename, delimiter='\t', skip_header=1)
  # For some reason AutoLab stores high frequency data first. Hence the flip.
  #data = np.flip(data, axis = 0)
  # Convert to a complex number
  f = data[:,0]
  Z = data[:, 1] + 1j*data[:,2]
  return f, Z

In [ ]:
def read_impedance_files (filenames):

  """
  Read multiple measurement text files and convert it into a 2D numpy array

  Parameters
  ----------
  filenames : list  
    Contains a list of strings with paths to each measurement file
  
  Returns
  ---------
  Z : numpy array
    impedance measurements
  f: numpy array
    frequencies
  """
  Z=[]
  f=[]
  for fname in filenames:
    f_, Z_ = readAutoLabFile(fname + '.txt')
    Z.append(Z_)
    f.append(f_)
    if (np.array(Z[0]).size != Z_.size):
       raise Exception('Data points in current measurement ({}) exceed the first measurement length ({})\n{}.'.format(Z_.size,np.array(Z[0]).size, fname))
  return np.array(f).T, np.array(Z).T

In [ ]:
def conductance(Z):
  """
  Converts impedance to conductance

  Parameters
  ----------
  Z : Numpy array with complex numbers 
    One impedance measurement in each column
  
  Returns
  ---------
  G : Numpy array
  """
  G = Z.real/(Z.real**2 + Z.imag**2)
  return G

In [ ]:
def cell_constant(freq,Z,ref_er,ref_conductivity):
  """
  Computes cell constant from the cell constant forumala

  Parameters
  ----------
  freq : Numpy array 
    frequencies of measurement in each column
  Z : Numpy array with complex numbers 
    One impedance measurement in each column 
  
  Returns
  ---------
  k : double
    cell constant
  """
  e0 = 8.8541878176e-12
  w = 2*np.pi*freq
  k = abs(Z)*np.sqrt(ref_conductivity**2+(w**2)*(e0**2)*(ref_er**2))
  return k

In [ ]:
def compute_conductivity(freq, Z, kappa):
  """
  Calculates conductivity based on the   value of kappa provided in parameters.

  Parameters
  ----------
  freq : numpy array
    frequencies 
  Z : Numpy array with complex numbers  
    One measurement in each column
  kappa : double
    cell constant
  
  Returns
  ---------

  """
  G= conductance(Z)
  conductivity = G/kappa
  return conductivity, G

In [ ]:
def read_impedance_data(filenames, group_name):
  """
  Reads impedance measurements 
  Packs data into a dictionary

  Parameters
  ----------
  
  Returns
  ---------

  """
  data = {}
  freq, Z = read_impedance_files(filenames)
  data['f'] = freq
  data['Z'] = Z
  data['group_name'] = group_name
  return data

In [ ]:
def read_conductivity_data(data, kappa):
  """
  Calculates conductivity based on the value of kappa provided in parameters.
  Packs data into a dictionary

  Parameters
  ----------
  
  Returns
  ---------

  """
  conductivity, G = compute_conductivity( data['f'],  data['Z'], Z, kappa)
  data['conductivity'] = conductivity
  data['G'] = G
  data['k'] = kappa
  data['sample_title'] = sample_title 
  return data

In [ ]:
def plot_mean_error(axes, x, y, xlabel, ylabel,label, color, alpha, plotfunc):
  """
  Takes a number of measurements stored in each column on numpy array y, 
  and plots mean and standard deviation in the same plot.

  Parameters
  ----------
  axes : numpy array
    frequencies measurement in each column
  x : Numpy array  
    frequencies of measurement in each column
  y : numpy array
    one measurement in each column
  Returns
  ---------

  """
  
  m = np.mean(y,axis = 1)
  e = np.std(y, axis = 1)
  plotfunc(x[:,1],m,'d', label=label, color = color)
  axes.fill_between(x[:,1],m+e,m-e, alpha = alpha, facecolor = color)
  #axes.text(x[0,0]-50000, max(m), str(round(m[0],3)), fontsize=16)
  #axes.text(x[-1,-1], max(m), str(round(m[-1],3)), fontsize=16)
  axes.set_xlabel(xlabel, fontsize = 16)
  axes.set_ylabel(ylabel, fontsize = 16)
  axes.tick_params(axis='both', which='major', labelsize=16)
  axes.grid(False)


In [ ]:
def plot_each_measurement(axes, x, y, xlabel, ylabel,label, color, alpha, plotfunc):
  """
  Takes a number of measurements stored in each column on numpy array y, 
  and plots mean and standard deviation in the same plot.

  Parameters
  ----------
  axes : numpy array
    frequencies measurement in each column
  x : Numpy array  
    frequencies of measurement in each column
  y : numpy array
    one measurement in each column
  Returns
  ---------

  """
  
  total_measurements = y.shape[1]
  for i in range(total_measurements):
    plotfunc(x[:,i],y[:,i],'.', label=label[i])
  axes.set_xlabel(xlabel, fontsize = 16)
  axes.set_ylabel(ylabel, fontsize = 16)
  axes.tick_params(axis='both', which='major', labelsize=16)
  axes.grid(False)

# Analyse standard liquid measurements

In [ ]:
def test_plot_mean_error():
  BASE_DATASET_PATH = '/content/drive/My Drive/Colab Notebooks/libs/bioimpedance/testdata/Saline 0.1/'
  #import os
  #from os import listdir
  #listdir(BASE_DATASET_PATH)
  filenames = ['saline0.1_temp20.3_second_001','saline0.1_temp20.3_second_002','saline0.1_temp20.3_second_003']
  group_name = 'NaCl-01'
  filenamespath = [BASE_DATASET_PATH + fname for fname in filenames]
  name = "Accent"
  cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
  color = iter(cmap.colors)  # type: list

  testdata = {}
  # Read data from measurement files
  testdata[group_name] = read_impedance_data(filenamespath, group_name)
  # Plot impedance data
  fig, ax = plt.subplots(figsize = (10,6))
  plot_mean_error(ax, testdata[group_name]['f'], abs(testdata[group_name]['Z']), 'f [Hz]','$|Z| (\Omega)$',testdata[group_name]['group_name'],next(color),0.5, ax.loglog )
  ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
  fig.suptitle(testdata[group_name]['group_name'] + ' - Impedance', fontsize = 16) 
  plt.show()

In [ ]:
#test_plot_mean_error()

In [ ]:
def test_plot_each_measurement():
  BASE_DATASET_PATH = '/content/drive/My Drive/Colab Notebooks/libs/bioimpedance/testdata/Saline 0.1/'
  #import os
  #from os import listdir
  #listdir(BASE_DATASET_PATH)
  filenames = ['saline0.1_temp20.3_second_001','saline0.1_temp20.3_second_002','saline0.1_temp20.3_second_003']
  group_name = 'NaCl-01'
  filenamespath = [BASE_DATASET_PATH + fname for fname in filenames]
  name = "Set1"
  cmap = get_cmap(name)  # type: matplotlib.colors.ListedColormap
  color = iter(cmap.colors)  # type: list

  testdata = {}
  # Read data from measurement files
  testdata[group_name] = read_impedance_data(filenamespath, group_name)
  # Plot impedance data
  fig, ax = plt.subplots(figsize = (10,6))
  plot_each_measurement(ax, testdata[group_name]['f'], abs(testdata[group_name]['Z']), 'f [Hz]','$|Z| (\Omega)$',filenames,next(color),1, ax.semilogx )
  plot_mean_error(ax, testdata[group_name]['f'], abs(testdata[group_name]['Z']), 'f [Hz]','$|Z| (\Omega)$','Mean, STD (shaded)',next(color),0.25, ax.loglog )
  ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
  fig.suptitle(testdata[group_name]['group_name'] + ' - Impedance', fontsize = 16) 
  plt.show()

In [ ]:
#test_plot_each_measurement()